# Imports

In [71]:
from datetime import datetime as dt

import json
import matplotlib.pyplot as plt

import numpy as np

import os

import pandas as pd
import pickle

pd.plotting.register_matplotlib_converters()
plt.rc("figure", figsize=(10, 8))
plt.rc("font", size=10)

## Date-Time Parsing Investigation

In [3]:
df = pd.read_csv("/workspaces/SE-Europe-Data_Challenge/data/raw/2022_to_2023/gen_DE_B01.csv")

In [4]:
df.drop(['EndTime', 'UnitName', 'AreaID', 'PsrType'], axis=1, inplace=True)
df.rename(columns={'StartTime': 'start_time', 'quantity': 'energy_generated'}, inplace=True)
df.sort_values(['start_time'], inplace=True)

In [5]:

d1 = dt.strptime("2021-12-31T23:45+00:00Z","%Y-%m-%dT%H:%M%zZ")

In [6]:
d1

datetime.datetime(2021, 12, 31, 23, 45, tzinfo=datetime.timezone.utc)

In [7]:
new_format = "%Y-%m-%d %H:%M:%S"

In [8]:
d1.strftime(new_format)

'2021-12-31 23:45:00'

In [49]:
type(d1)

datetime.datetime

## EDA Training Data

In [21]:
data = pd.read_csv("/workspaces/SE-Europe-Data_Challenge/data/processed.csv", parse_dates= ['start_time'], index_col='start_time')

In [22]:
data.head()

,green_energy_DE,DE_Load,green_energy_HU,HU_Load,green_energy_IT,IT_Load,green_energy_PO,PO_Load,green_energy_SP,SP_Load,green_energy_UK,UK_Load,green_energy_DK,DK_Load,green_energy_SE,SE_Load,green_energy_NE,NE_Load
start_time,,,,,,,,,,,,,,,,,,
2021-12-31 23:00:00,39401,42196,285,4254,5801.0,21121.0,4641.0,14438.0,11244,20827,0.0,677.0,3652.0,3314.0,11060.0,15331.0,3554,10326
2022-01-01 00:00:00,157373,165125,1376,16457,5745.0,19756.0,4491.0,13935.0,10827,19530,0.0,1244.0,3605.0,3218.0,11107.0,15331.0,12768,40706
2022-01-01 01:00:00,151747,160415,1526,15426,6228.0,18685.0,4436.0,13579.0,11140,18383,0.0,1131.0,3309.0,3126.0,11036.0,15270.0,11131,39465
2022-01-01 02:00:00,147938,158035,1560,14781,6407.0,18124.0,4568.0,13397.0,11361,17680,0.0,1091.0,3043.0,3080.0,10509.0,15150.0,11324,38923
2022-01-01 03:00:00,141816,157016,1287,14630,5760.0,18400.0,4559.0,13364.0,11247,17396,0.0,969.0,2852.0,3044.0,10770.0,15387.0,11290,38211


In [23]:
data.columns

Index(['green_energy_DE', 'DE_Load', 'green_energy_HU', 'HU_Load',
       'green_energy_IT', 'IT_Load', 'green_energy_PO', 'PO_Load',
       'green_energy_SP', 'SP_Load', 'green_energy_UK', 'UK_Load',
       'green_energy_DK', 'DK_Load', 'green_energy_SE', 'SE_Load',
       'green_energy_NE', 'NE_Load'],
      dtype='object')

In [24]:
data.shape

(8761, 18)

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8761 entries, 2021-12-31 23:00:00 to 2022-12-31 23:00:00
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   green_energy_DE  8761 non-null   int64  
 1   DE_Load          8761 non-null   int64  
 2   green_energy_HU  8761 non-null   int64  
 3   HU_Load          8761 non-null   int64  
 4   green_energy_IT  8761 non-null   float64
 5   IT_Load          8761 non-null   float64
 6   green_energy_PO  8761 non-null   float64
 7   PO_Load          8761 non-null   float64
 8   green_energy_SP  8761 non-null   int64  
 9   SP_Load          8761 non-null   int64  
 10  green_energy_UK  8761 non-null   float64
 11  UK_Load          8761 non-null   float64
 12  green_energy_DK  8761 non-null   float64
 13  DK_Load          8761 non-null   float64
 14  green_energy_SE  8761 non-null   float64
 15  SE_Load          8761 non-null   float64
 16  green_energy_NE  8761 no

In [26]:
for column in data.columns:
    if column != 'start_time':  # Skipping the 'start_time' column
        data[column] = data[column].astype(int)

In [27]:
data.dtypes

green_energy_DE    int64
DE_Load            int64
green_energy_HU    int64
HU_Load            int64
green_energy_IT    int64
IT_Load            int64
green_energy_PO    int64
PO_Load            int64
green_energy_SP    int64
SP_Load            int64
green_energy_UK    int64
UK_Load            int64
green_energy_DK    int64
DK_Load            int64
green_energy_SE    int64
SE_Load            int64
green_energy_NE    int64
NE_Load            int64
dtype: object

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8761 entries, 2021-12-31 23:00:00 to 2022-12-31 23:00:00
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   green_energy_DE  8761 non-null   int64
 1   DE_Load          8761 non-null   int64
 2   green_energy_HU  8761 non-null   int64
 3   HU_Load          8761 non-null   int64
 4   green_energy_IT  8761 non-null   int64
 5   IT_Load          8761 non-null   int64
 6   green_energy_PO  8761 non-null   int64
 7   PO_Load          8761 non-null   int64
 8   green_energy_SP  8761 non-null   int64
 9   SP_Load          8761 non-null   int64
 10  green_energy_UK  8761 non-null   int64
 11  UK_Load          8761 non-null   int64
 12  green_energy_DK  8761 non-null   int64
 13  DK_Load          8761 non-null   int64
 14  green_energy_SE  8761 non-null   int64
 15  SE_Load          8761 non-null   int64
 16  green_energy_NE  8761 non-null   int64
 17  NE_Load         

In [29]:
data.isnull().sum()

green_energy_DE    0
DE_Load            0
green_energy_HU    0
HU_Load            0
green_energy_IT    0
IT_Load            0
green_energy_PO    0
PO_Load            0
green_energy_SP    0
SP_Load            0
green_energy_UK    0
UK_Load            0
green_energy_DK    0
DK_Load            0
green_energy_SE    0
SE_Load            0
green_energy_NE    0
NE_Load            0
dtype: int64

In [30]:
processed_df = data.copy(deep=True)

In [31]:
processed_df.index

DatetimeIndex(['2021-12-31 23:00:00', '2022-01-01 00:00:00',
               '2022-01-01 01:00:00', '2022-01-01 02:00:00',
               '2022-01-01 03:00:00', '2022-01-01 04:00:00',
               '2022-01-01 05:00:00', '2022-01-01 06:00:00',
               '2022-01-01 07:00:00', '2022-01-01 08:00:00',
               ...
               '2022-12-31 14:00:00', '2022-12-31 15:00:00',
               '2022-12-31 16:00:00', '2022-12-31 17:00:00',
               '2022-12-31 18:00:00', '2022-12-31 19:00:00',
               '2022-12-31 20:00:00', '2022-12-31 21:00:00',
               '2022-12-31 22:00:00', '2022-12-31 23:00:00'],
              dtype='datetime64[ns]', name='start_time', length=8761, freq=None)

In [19]:
# Calculating the surplus for each country and creating the label column

country_ids = {
    "SP": 0,  # Spain
    "UK": 1,  # United Kingdom
    "DE": 2,  # Germany
    "DK": 3,  # Denmark
    "SE": 4,  # Sweden
    "HU": 5,  # Hungary
    "IT": 6,  # Italy
    "PO": 7,  # Poland
    "NE": 8   # Netherlands
}



In [20]:
# Calculate surplus for each country
for country_code in country_ids.keys():
    green_energy_col = f'green_energy_{country_code}'
    load_col = f'{country_code}_Load'
    surplus_col = f'{country_code}_surplus'
    processed_df[surplus_col] = processed_df[green_energy_col] - processed_df[load_col]

In [21]:
# Determine the country with the highest surplus for each hour
surplus_cols = [f'{code}_surplus' for code in country_ids.keys()]
processed_df['label'] = processed_df[surplus_cols].idxmax(axis=1)
processed_df['label'] = processed_df['label'].apply(lambda x: country_ids[x.split('_')[0]])

In [22]:
processed_df.shape

(8761, 29)

In [23]:
processed_df.isnull().sum()

start_time         0
green_energy_DE    0
DE_Load            0
green_energy_HU    0
HU_Load            0
green_energy_IT    0
IT_Load            0
green_energy_PO    0
PO_Load            0
green_energy_SP    0
SP_Load            0
green_energy_UK    0
UK_Load            0
green_energy_DK    0
DK_Load            0
green_energy_SE    0
SE_Load            0
green_energy_NE    0
NE_Load            0
SP_surplus         0
UK_surplus         0
DE_surplus         0
DK_surplus         0
SE_surplus         0
HU_surplus         0
IT_surplus         0
PO_surplus         0
NE_surplus         0
label              0
dtype: int64

In [24]:
# Correcting the filtering process for the year 2022

processed_df['start_time'] = pd.to_datetime(processed_df['start_time'])

processed_df_2022 = processed_df[processed_df['start_time'].dt.year == 2022]

# Displaying the first few rows of the filtered DataFrame
processed_df_2022.head()

,start_time,green_energy_DE,DE_Load,green_energy_HU,HU_Load,green_energy_IT,IT_Load,green_energy_PO,PO_Load,green_energy_SP,...,SP_surplus,UK_surplus,DE_surplus,DK_surplus,SE_surplus,HU_surplus,IT_surplus,PO_surplus,NE_surplus,label
1,2022-01-01 00:00:00,157373,165125,1376,16457,5745,19756,4491,13935,10827,...,-8703,-1244,-7752,387,-4224,-15081,-14011,-9444,-27938,3
2,2022-01-01 01:00:00,151747,160415,1526,15426,6228,18685,4436,13579,11140,...,-7243,-1131,-8668,183,-4234,-13900,-12457,-9143,-28334,3
3,2022-01-01 02:00:00,147938,158035,1560,14781,6407,18124,4568,13397,11361,...,-6319,-1091,-10097,-37,-4641,-13221,-11717,-8829,-27599,3
4,2022-01-01 03:00:00,141816,157016,1287,14630,5760,18400,4559,13364,11247,...,-6149,-969,-15200,-192,-4617,-13343,-12640,-8805,-26921,3
5,2022-01-01 04:00:00,131970,154552,1309,14688,5501,19223,4374,13449,10868,...,-6676,-896,-22582,-406,-4885,-13379,-13722,-9075,-27053,3


In [29]:
processed_df_2022.label.unique()

array([3, 1, 0, 2, 4])

In [25]:
processed_df_2022.shape

(8760, 29)

In [40]:
processed_df_2022.set_index("start_time", inplace=True)

In [41]:
processed_df_2022

,green_energy_DE,DE_Load,green_energy_HU,HU_Load,green_energy_IT,IT_Load,green_energy_PO,PO_Load,green_energy_SP,SP_Load,...,SP_surplus,UK_surplus,DE_surplus,DK_surplus,SE_surplus,HU_surplus,IT_surplus,PO_surplus,NE_surplus,label
start_time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,157373,165125,1376,16457,5745,19756,4491,13935,10827,19530,...,-8703,-1244,-7752,387,-4224,-15081,-14011,-9444,-27938,3
2022-01-01 01:00:00,151747,160415,1526,15426,6228,18685,4436,13579,11140,18383,...,-7243,-1131,-8668,183,-4234,-13900,-12457,-9143,-28334,3
2022-01-01 02:00:00,147938,158035,1560,14781,6407,18124,4568,13397,11361,17680,...,-6319,-1091,-10097,-37,-4641,-13221,-11717,-8829,-27599,3
2022-01-01 03:00:00,141816,157016,1287,14630,5760,18400,4559,13364,11247,17396,...,-6149,-969,-15200,-192,-4617,-13343,-12640,-8805,-26921,3
2022-01-01 04:00:00,131970,154552,1309,14688,5501,19223,4374,13449,10868,17544,...,-6676,-896,-22582,-406,-4885,-13379,-13722,-9075,-27053,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-31 19:00:00,175631,184106,851,17488,4198,24809,6959,15211,67056,104484,...,-37428,824,-8475,-1948,-4261,-16637,-20611,-8252,-28405,1
2022-12-31 20:00:00,176499,175401,888,17075,3676,23169,6910,14641,62432,97124,...,-34692,830,1098,-1986,-4615,-16187,-19493,-7731,-26192,2
2022-12-31 21:00:00,175678,168830,807,16717,3641,21857,7054,13977,61096,87316,...,-26220,676,6848,-1732,-4565,-15910,-18216,-6923,-24637,2


In [43]:
processed_df_2022.to_csv("data/final_processed_2022.csv")

In [44]:
processed_df_2022.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2022-01-01 00:00:00 to 2022-12-31 23:00:00
Data columns (total 28 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   green_energy_DE  8760 non-null   int64
 1   DE_Load          8760 non-null   int64
 2   green_energy_HU  8760 non-null   int64
 3   HU_Load          8760 non-null   int64
 4   green_energy_IT  8760 non-null   int64
 5   IT_Load          8760 non-null   int64
 6   green_energy_PO  8760 non-null   int64
 7   PO_Load          8760 non-null   int64
 8   green_energy_SP  8760 non-null   int64
 9   SP_Load          8760 non-null   int64
 10  green_energy_UK  8760 non-null   int64
 11  UK_Load          8760 non-null   int64
 12  green_energy_DK  8760 non-null   int64
 13  DK_Load          8760 non-null   int64
 14  green_energy_SE  8760 non-null   int64
 15  SE_Load          8760 non-null   int64
 16  green_energy_NE  8760 non-null   int64
 17  NE_Load         

In [45]:
processed_df_2022.columns

Index(['green_energy_DE', 'DE_Load', 'green_energy_HU', 'HU_Load',
       'green_energy_IT', 'IT_Load', 'green_energy_PO', 'PO_Load',
       'green_energy_SP', 'SP_Load', 'green_energy_UK', 'UK_Load',
       'green_energy_DK', 'DK_Load', 'green_energy_SE', 'SE_Load',
       'green_energy_NE', 'NE_Load', 'SP_surplus', 'UK_surplus', 'DE_surplus',
       'DK_surplus', 'SE_surplus', 'HU_surplus', 'IT_surplus', 'PO_surplus',
       'NE_surplus', 'label'],
      dtype='object')

In [48]:
processed_df_2022 = pd.read_csv("/workspaces/SE-Europe-Data_Challenge/data/final_processed_2022.csv", parse_dates=['start_time'], index_col='start_time')

In [49]:
processed_df_2022.shape

(8760, 28)

## Training and Testing Split

In [50]:
# Splitting the dataset into training and testing sets

# Since this is time-series data, we'll perform a chronological split (not random)
# We'll use the last 20% of the data as the test set

split_index = int(len(processed_df_2022) * 0.8)
train_df = processed_df_2022.iloc[:split_index]
test_df = processed_df_2022.iloc[split_index:]

# Verifying the split
train_test_split_info = {
    "Total Data Points": len(processed_df_2022),
    "Training Data Points": len(train_df),
    "Testing Data Points": len(test_df),
    "Split Index": split_index
}

train_test_split_info

{'Total Data Points': 8760,
 'Training Data Points': 7008,
 'Testing Data Points': 1752,
 'Split Index': 7008}

In [69]:
train_df.to_csv("data/train.csv")


In [70]:
test_df.to_csv("data/test.csv")

## Training the Model

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

X_train = train_df.drop('label', axis=1)
y_train = train_df['label']

X_test = test_df.drop('label', axis=1)
y_test = test_df['label']

In [53]:
# Initialize and train the RandomForestClassifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [54]:
# Predict on the test set
predictions = model.predict(X_test)

In [55]:
# Calculate the F1 score
f1_score_value = f1_score(y_test, predictions, average='macro')
f1_score_value

0.5491424425426029

### Saving the Model

In [68]:
model_file_path = 'models/random_forest_model.pkl'

# Save the model to a pickle file
with open(model_file_path, 'wb') as file:
    pickle.dump(model, file)


### Saving the Predictions

In [59]:
# Adjusting the format of the JSON to include a "target" property

# Creating a dictionary with 'target' as the key and the predictions as the value
predictions_json = {'target': {str(index): int(prediction) for index, prediction in enumerate(predictions)}}

# Convert the formatted predictions to JSON
predictions_json = json.dumps(predictions_json)

# Saving the formatted JSON to a file
json_file_path = 'predictions/predictions.json'
with open(json_file_path, 'w') as file:
    file.write(predictions_json)

json_file_path


'predictions/predictions.json'

In [61]:
with open(json_file_path, 'r') as file:
    predictions_json = json.load(file)

# Getting the length of the predictions list
predictions_length = len(predictions_json["target"])
predictions_length

1752

In [66]:
# def basic_exploration(file_path):
#     df = pd.read_csv(file_path)
#     print(f"""\nThe column names in the data are: \n{df.columns}\n{'x' * 60}
#           The datatypes of the columns are: \n{df.dtypes}
#           {'x' * 60}
#           The minimum value of StartTime: {df['StartTime'].min()}
#           The maximum value of StartTime: {df['StartTime'].max()}
#           The minimum value of EndTime: {df['EndTime'].min()}
#           The maximum value of EndTime: {df['EndTime'].max()}
#           The minimum value of energy generated:  {df['quantity'].min()}
#           The maximum value of energy generated: {df['quantity'].max()}
#           """)

In [67]:
# basic_exploration("/workspaces/SE-Europe-Data_Challenge/data/processed/2022_to_2023/gen_UK_B19_modified.csv")